# Distributed Data Classification with NeMo Curator's `FineWebEduClassifier`

This notebook demonstrates the use of NeMo Curator's `FineWebEduClassifier`. The [FineWeb-Edu classifier](https://huggingface.co/HuggingFaceFW/fineweb-edu-classifier) is used for judging the educational value of web pages. It helps with data annotation, which is useful in data blending for foundation model training. Please refer to the Hugging Face page for more information about the FineWeb-Edu classifier here: https://huggingface.co/HuggingFaceFW/fineweb-edu-classifier.

This tutorial requires at least 1 NVIDIA GPU with:
  - Volta™ or higher (compute capability 7.0+)
  - CUDA 12.x

For more information about the classifiers, refer to our [Distributed Data Classification](https://docs.nvidia.com/nemo/curator/latest/curate-text/process-data/quality-assessment/distributed-classifier.html) documentation page.

Before running this notebook, see this [Installation Guide](https://docs.nvidia.com/nemo/curator/latest/admin/installation.html#admin-installation) page for instructions on how to install NeMo Curator. Be sure to use an installation method which includes GPU dependencies (`text_cuda12` or `all`). Check proper installation with:

In [ ]:
# First, silence Curator logs via Loguru
import os

os.environ["LOGURU_LEVEL"] = "ERROR"

In [ ]:
import nemo_curator

nemo_curator.__version__  # should be >= 1.0.0

We can check that GPUs are available, then check that the `gpustat` dependency was installed:

In [ ]:
!nvidia-smi

In [ ]:
import gpustat

gpustat.__version__  # check gpu dependency is installed

The following imports are required for this tutorial:

In [2]:
import pandas as pd

from nemo_curator.core.client import RayClient
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.text.classifiers import FineWebEduClassifier
from nemo_curator.stages.text.io.reader.jsonl import JsonlReader
from nemo_curator.stages.text.io.reader.parquet import ParquetReader
from nemo_curator.stages.text.io.writer.jsonl import JsonlWriter
from nemo_curator.stages.text.io.writer.parquet import ParquetWriter
from nemo_curator.utils.file_utils import get_all_file_paths_under

To run a pipeline in NeMo Curator, we must start a Ray cluster. This can be done manually (see the [Ray documentation](https://docs.ray.io/en/latest/ray-core/starting-ray.html)) or with Curator's `RayClient`:

In [3]:
try:
    ray_client = RayClient()
    ray_client.start()
except Exception as e:
    msg = f"Error initializing Ray client: {e}"
    raise RuntimeError(msg) from e

# Initialize Read, Classification, and Write Stages

Functions in NeMo Curator are called stages. For this tutorial, we will initialize 3 stages: a file reader, a FineWeb-Edu classification stage, and a file writer.

For this tutorial, an existing directory containing JSONL or Parquet files may be provided via `input_file_path`. If `input_file_path` does not exist or is empty, we download a small dataset from Hugging Face.

In [4]:
input_file_path = "./input_data_dir"
input_file_type = "parquet"  # can be "jsonl" or "parquet"
text_field = "text"

if len(get_all_file_paths_under(input_file_path)) == 0:
    from huggingface_hub import snapshot_download

    # Download Parquet files from Hugging Face
    # This dataset contains only English texts
    snapshot_download(
        repo_id="HuggingFaceFW/fineweb-edu-llama3-annotations",
        repo_type="dataset",
        local_dir=input_file_path,
        allow_patterns=["data/*"],
    )
    print(f"Downloaded Hugging Face dataset to {input_file_path}")

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Downloaded Hugging Face dataset to ./input_data_dir


We can define the reader stage with:

In [5]:
# Read directory of files and only read the text field
if input_file_type == "jsonl":
    read_stage = JsonlReader(input_file_path, files_per_partition=1, fields=[text_field])
elif input_file_type == "parquet":
    read_stage = ParquetReader(input_file_path, files_per_partition=1, fields=[text_field])
else:
    msg = f"Invalid input file type: {input_file_type}"
    raise ValueError(msg)

The classifier stage is broken down under the hood into a tokenizer stage and a model inference stage. Tokenization is run on the CPU while model inference is run on the GPU. This means that behind the scenes, the `FineWebEduClassifier` stage is actually being broken down into 2 stages (some parameters and details omitted to avoid complexity, please refer to the documentation for more details):

```python
class TokenizerStage:
    self.resources = Resources(cpus=1)
    self.model_identifier = "HuggingFaceFW/fineweb-edu-classifier"
    self.text_field = "text"
    self.padding_side = "right"
    ...
class ModelStage:
    self.resources = Resources(cpus=1, gpus=1)
    self.model_identifier = "HuggingFaceFW/fineweb-edu-classifier"
    self.model_inference_batch_size = 256
    ...
```

Optionally, the classifier predictions may be filtered to include only texts with values listed in `filter_by`. If the `filter_by` parameter is set, then a third stage is added:

```python
def filter_by_category(self, value: str) -> bool:
    return value in self.filter_by

...

if self.filter_by is not None and len(self.filter_by) > 0:
    self.stages.append(Filter(filter_fn=self.filter_by_category, filter_field=...))
```

Since the FineWeb-Edu classifier outputs a floating point value ranging from 0 to 5, Curator labels samples with a value of 2.5 or higher as `high_quality` and samples with a value lower than 2.5 as `low_quality`.

See the [API Reference](https://docs.nvidia.com/nemo/curator/latest/apidocs/stages/stages.text.classifiers.fineweb_edu.html#stages.text.classifiers.fineweb_edu.FineWebEduClassifier) for more information about the `FineWebEduClassifier` class.

In [6]:
# Initialize the FineWeb-Edu classifier
classifier_stage = FineWebEduClassifier(text_field=text_field)

# If desired, you may filter your dataset with:
# classifier_stage = FineWebEduClassifier(filter_by=["high_quality"])  # noqa: ERA001
# or
# classifier_stage = FineWebEduClassifier(filter_by=["low_quality"])  # noqa: ERA001

Finally, we can define a stage for writing the results:

In [7]:
output_file_path = "./fineweb_edu_classifier_results"
output_file_type = "parquet"  # can be "jsonl" or "parquet"

# Use mode="overwrite" to overwrite the output directory if it already exists
# This helps to ensure that the correct output is written
if output_file_type == "jsonl":
    write_stage = JsonlWriter(output_file_path, mode="overwrite")
elif output_file_type == "parquet":
    write_stage = ParquetWriter(output_file_path, mode="overwrite")
else:
    msg = f"Invalid output file type: {output_file_type}"
    raise ValueError(msg)

2025-12-03 14:48:50,438	INFO usage_lib.py:447 -- Usage stats collection is disabled.
2025-12-03 14:48:50,438	INFO scripts.py:914 -- Local node IP: 127.0.1.1


# Initialize Pipeline

In NeMo Curator, we use pipelines to run distributed data workflows using Ray. Pipelines take care of resource allocation and autoscaling to achieve enhanced performance and minimize GPU idleness.

For the distributed data classifiers, we are able to achieve speedups by ensuring that model inference is run in parallel across all available GPUs, while other stages such as I/O, tokenization, and filtering are run across all available CPUs. This is possible because Curator pipelines are composable, which allows each stage in a pipeline to run independently and with its own specified hardware resources.

In [8]:
classifier_pipeline = Pipeline(name="classifier_pipeline", description="Run a classifier pipeline")

# Add stages to the pipeline
classifier_pipeline.add_stage(read_stage)
classifier_pipeline.add_stage(classifier_stage)
classifier_pipeline.add_stage(write_stage)

Pipeline(name='classifier_pipeline', stages=[parquet_reader(ParquetReader), fineweb_edu_classifier_classifier(FineWebEduClassifier), parquet_writer(ParquetWriter)])

Composability is also what allows a classifier to sit between pre-processing and post-processing stages. Typical text pre-processing add-ons include text normalization (lowercasing, URL/email removal, Unicode cleanup) and language identification and filtering (to keep only target languages). A full pipeline may look something like:

```python
pipeline = Pipeline(name="full_pipeline")
pipeline.add_stage(read_stage)                # reader (JSONL/Parquet)
pipeline.add_stage(lang_id_stage)             # optional: language filter
pipeline.add_stage(classifier_stage)          # classifier
pipeline.add_stage(write_stage)               # writer (JSONL/Parquet)
```

# Run the  Classifier

Let's run the full classifier pipeline:

In [9]:
# Run the pipeline
result = classifier_pipeline.run()

2025-12-03 14:48:50,820	INFO worker.py:1692 -- Using address 127.0.1.1:6381 set in the environment variable RAY_ADDRESS
2025-12-03 14:48:50,823	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6381...


2025-12-03 14:48:52,559	SUCC scripts.py:950 -- --------------------
2025-12-03 14:48:52,559	SUCC scripts.py:951 -- Ray runtime started.
2025-12-03 14:48:52,559	SUCC scripts.py:952 -- --------------------
2025-12-03 14:48:52,559	INFO scripts.py:954 -- Next steps
2025-12-03 14:48:52,559	INFO scripts.py:957 -- To add another node to this Ray cluster, run
2025-12-03 14:48:52,559	INFO scripts.py:960 --   ray start --address='127.0.1.1:6381'
2025-12-03 14:48:52,559	INFO scripts.py:969 -- To connect to this Ray cluster:
2025-12-03 14:48:52,559	INFO scripts.py:971 -- import ray
2025-12-03 14:48:52,559	INFO scripts.py:972 -- ray.init(_node_ip_address='127.0.1.1')
2025-12-03 14:48:52,560	INFO scripts.py:984 -- To submit a Ray job using the Ray Jobs CLI:
2025-12-03 14:48:52,560	INFO scripts.py:985 --   RAY_API_SERVER_ADDRESS='http://127.0.0.1:8267' ray job submit --working-dir . -- python my_script.py
2025-12-03 14:48:52,560	INFO scripts.py:994 -- See https://docs.ray.io/en/latest/cluster/running

2025-12-03 14:48:52,848	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8267 
/raid/syurick/conda/ray-curator/lib/python3.12/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-12-03 14:48:53,797	INFO worker.py:1692 -- Using address 127.0.1.1:6381 set in the environment variable RAY_ADDRESS
2025-12-03 14:48:53,798	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 127.0.1.1:6381...
2025-12-03 14:48:53,799	INFO worker.py:1851 -- Calling ray.init() again after it has already been called.
Fetching 15 files: 100%|██████████| 15/15 [00:00<00:00, 50493.23it/s]


Since the pipeline ran to completion and the result was written to disk, we can shut down the Ray cluster with:

In [10]:
try:
    ray_client.stop()
except Exception as e:  # noqa: BLE001
    print(f"Error stopping Ray client: {e}")

# Inspect the Output

The write stage returns a list of written files. We can read the output file as a Pandas DataFrame for inspection.

In [11]:
# For simplicity, we take the first written file from the writer stage
# In real pipelines, adjust as needed
result_file = result[0].data[0]

result_df = pd.read_json(result_file, lines=True) if output_file_type == "jsonl" else pd.read_parquet(result_file)
result_df.head()

,text,fineweb-edu-score-float,fineweb-edu-score-int,fineweb-edu-score-label
0,Schiaparelli Haute Couture F/W 2022 : Daniel R...,0.789551,1,low_quality
1,The Bellows AFB Cabins are awesome and on the ...,0.452148,0,low_quality
2,I recently had the opportunity to participate ...,0.710938,1,low_quality
3,- Prospective Students - Current Students - Ab...,2.535156,3,high_quality
4,"For this Big Sky Town Center home, we partnere...",0.632812,1,low_quality


Let's examine the distribution of predictions for this file:

In [12]:
result_df["fineweb-edu-score-label"].value_counts()

fineweb-edu-score-label
low_quality     54002
high_quality     4426
Name: count, dtype: int64

We can see that the predictions were generated as expected.